## Init & utils

In [ ]:
%%bash
pip install --upgrade pip
pip install jsonpath-rw
pip install jsonpath-rw-ext
pip install jsondiff
pip install seaborn

In [ ]:
import json 
from json import JSONEncoder, JSONDecoder
import pickle

class PythonObjectEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (list, dict, str, unicode, int, float, bool, type(None))):
            return JSONEncoder.default(self, obj)
        return {'_python_object': pickle.dumps(obj)}

def as_python_object(dct):
    if '_python_object' in dct:
        return pickle.loads(str(dct['_python_object']))
    return dct

def save_to_json_file(filename, content):
    temp = json.dumps(content, cls=PythonObjectEncoder)
    return save_to_file(filename, temp)

def save_to_file(filename, content):
    with open(filename, 'w') as file:
        file.write(content)
    return "saved " + filename    

def load_from_json_file(file_name):
    content = load_from_file(file_name)
    return json.load(content, object_hook=as_python_object)

def load_from_file(file_name):
    return open(file_name)

In [ ]:
import copy

def inc_dict_val(d, k, v):
    d[k] = int(d.get(k, 0)) + int(v)

def alter_dict(D, **F):
    result = copy.deepcopy(D)    
    result.update(**F)
    return result   

def map_dict(src, *args, **kvargs):
    dst = dict()
    for v in args:
        dst[v] = src[v]
    for k,v in kvargs.items():
        dst[k] = src[v]
    return dst

def remove_from_dict(d, key):
    if key in d:
        del d[key]

def inc_dict_value(d, prop, delta=1):
    d[prop] = d.get(prop, 0) + int(delta)

## Printing

In [ ]:
from pprint import pprint
from IPython.display import JSON
width = 180

In [ ]:
import re

def print_log(log, consumer=print, **args):
    if log:
        params = {g:'' for g in re.findall('\$\{([^}]+)\}', log)}
        params.update(args)
        consumer(log.replace('$', '').format(**params))

In [ ]:
from jsondiff import diff

def print_diff(old_dict, new_dict, consumer=print):     
    consumer(get_diff(old_dict, new_dict))
    
def get_diff(old_dict, new_dict):     
    return diff(old_dict, new_dict, syntax='symmetric')

## Generic game functions

In [ ]:
def load_game(fn):
    return load_from_json_file(fn)['data']['data']['data']

In [ ]:
def get_step(game, move_id=0, sub_move_id=0, **kvargs):
    return game[move_id]['data'][sub_move_id]

In [ ]:
def move_to_next_step(game, game_state):
    move_id     = game_state.get('move_id', 0)
    sub_move_id = game_state.get('sub_move_id', 0) + 1
    
    if len(game[move_id]['data']) <= sub_move_id:
        move_id    += 1
        sub_move_id = 0
    
    return alter_dict(game_state,
        move_id     = move_id,
        sub_move_id = sub_move_id
    )

In [ ]:
def default_action(game_state, step):
    print("Unknown action '%s' :(" % step['type'])
    display(JSON({'state': game_state, 'step': step}, indent=4))
    return None

In [ ]:
def apply_step(game_state, step):
    if not step['args']: 
        step['args'] = dict()
    elif not isinstance(step['args'], dict): 
        step['args'] = {str(i):v for i,v in enumerate(step['args'])}
    print_log(step['log'], **step['args'])
    a = actions.get(step['type'], None)
    if a:
        return a(game_state, **step['args'])
    else:
        return default_action(game_state, step)        

## Puerto rico game functions

## initialization

In [ ]:
good_types = ['corn', 'indigo', 'sugar', 'tobacco', 'coffee']

def setup_buildings(buildings, **kvargs):
    result = {}
    def inc(d, k): d[k] = 1 + d.get(k, 0) 
    for id,b in buildings.items():
        inc(result, b['bld_type'])
    return result

def setup_plantations(plantations_to_set, **kvargs):
    result = [p['res_type'] for id,p in plantations_to_set.items()]
    return result

def setup_ships(ships, **kvargs): 
    result = {v['shp_capacity']: dict() for k,v in ships.items()}
    return result

def setup_stock(**kvargs):
    result = []
    return result

def setup_VPs(counters, **kvargs): 
    return int(counters['shipping_points_bank']['counter_value'])

def setup_bank(counters, **kvargs): 
    return int(counters['doubloons_bank']['counter_value'])

def setup_workers(counters, **kvargs): 
    result = {
        'available': int(counters['colonists_bank']['counter_value']),
        'arriving':  int(counters['colonists_ship']['counter_value'])
    }
    return result

def setup_goods(counters, **kvargs): 
    result = {
        'coffee'  : int(counters['resource_coffee_bank']['counter_value']),
        'corn'    : int(counters['resource_corn_bank']['counter_value']),
        'indigo'  : int(counters['resource_indigo_bank']['counter_value']),
        'sugar'   : int(counters['resource_sugar_bank']['counter_value']),
        'tobacco' : int(counters['resource_tobacco_bank']['counter_value']),
    }
    return result

def setup_roles(role_tooltips, **kvargs):
    result = {k:{'money': v['money'], 'rol_type': v['rol_type']} for k,v in role_tooltips.items()}
    return result
    
def setup_playerorder(playerorder, **kvargs):
    result = [str(v) for v in playerorder]
    return result

def setup_player(index, username, money, plantaion, **kvargs):
    result = {
        '_': username,
        'money': int(money),
        'building_vp': 0,
        'delivery_vp': 0,
        'buildings': {'commercial': {}, 'factories': {g:{'jobs':0, 'workers':0} for g in good_types}},
        'goods': {g:0 for g in good_types},
        'plantations': {g:1 if g == plantaion else 0 for g in (good_types + ['rock'])},
        'potential_vp': {'commercial': 0, 'factories': 0, 'workers': 0},
        'slots': {'buildings': 0, 'plantations': 1},
        'workers': {'total': 0, 'unemployed': 0}
    }
    return result

def setup_players(players, playerorder, counters, plantations_owned, **kvargs):
    result = {}
    for index,id in enumerate(playerorder):
        id = str(id)
        money = counters['doubloons_player_'+id]['counter_value']
        plantation = [v['res_type'] for k,v in plantations_owned.items() if v['player_id'] == id][0]
        result[id] = setup_player(index, players[id]['name'], money, plantation)
    return result

def setup_table(datas):
    r = {
        'buildings': setup_buildings(**datas),
        'plantations': setup_plantations(**datas),
        'ships': setup_ships(**datas),
        'stock': setup_stock(**datas),
        'VPs': setup_VPs(**datas),
        'money': setup_bank(**datas),
        'workers': setup_workers(**datas),
        'goods': setup_goods(**datas),
        'roles': setup_roles(**datas),
        'playerorder': setup_playerorder(**datas)
    }
    return r

def setup(datas):
    return {
        'move_id': 0,
        'sub_move_id': 0,
        'table': setup_table(datas),
        'players': setup_players(**datas),
        'governor': datas['governor']
    }

In [ ]:
def setup_lookups(buildings, plantations_owned, **kvargs):
    global building_lookup
    building_lookup = {k:{
        'name': b['bld_type'],
        'type': 'factories' if b['res_type'] else 'commercial',
        'res_type': b['res_type'],
        'potential_vp': 1 if b['bld_valence'] == "1" else 2,
        'tile_size': 2 if b['bld_max_discount'] == "4" else 1,
        'jobs': int(b['bld_valence']),
        'cost': int(b['bld_cost']),
        'max_discount': int(b['bld_max_discount']),
        'vp': int(b['bld_score'])
    } for k,b in buildings.items()}
    
    global plantation_lookup
    plantation_lookup = {k:p['res_type'] for k,p in plantations_owned.items()}

In [ ]:
new_game_state = 1
if not 'actions' in globals():
    global actions
    actions = dict()    

game_log = load_game('table_27174212.json')
datas = load_from_json_file('datas_27174212.json')[7]
game_state = setup(datas)
setup_lookups(**datas)

### Unverified

In [ ]:
def do_nothing(log=None):
    def dn(state, **args):
        print_log(log, **args)
        return state
    return dn
log = ' - d:${description:70} t:${type}, n:${name}'#', a:${action}'#', possible_actions:${possibleactions}'

actions['leaveGameState']      = do_nothing('<-[leaveGameState ]' + log)
actions['gameStateChange']     = do_nothing('->[gameStateChange]' + log)
actions['updateReflexionTime'] = do_nothing('  [updateReflexionTime]')
actions['message' ]            = do_nothing('  [message]')
actions['gameStateMultipleActiveUpdate'] = do_nothing('  gameStateMultipleActiveUpdate')

In [ ]:
def draftCompleted(state, **args):
    return state

actions['draftCompleted'] = draftCompleted

In [ ]:
def roleSelected(state, rol_id, rol_type, unavailable_roles, **args):
    new_state = alter_dict(state)
    new_state['table']['roles'][rol_id]['selected'] = True
    return new_state

actions['roleSelected'] = roleSelected

In [ ]:
def plantationSettlement(state, player_id, **args):
    new_state = alter_dict(state)
    pl = get_player(new_state, player_id)
#     update_counter(new_state['table'], **args)
    pick_field(new_state['table'], pl, **args)
    return new_state

def get_player(state, player_id, **args):
    return state['players'][str(player_id)]

def pick_field(table, player, res_type, pla_id, **args):
    player['plantations'][res_type] = 1 + player['plantations'][res_type]
    inc_dict_value(player['slots'], 'plantations', 1)
    if res_type in table['plantations']:
        table['plantations'].remove(res_type)
    plantation_lookup[pla_id] = res_type


def update_counter(target, counter_name=None, counter_delta=0, **args):
    if counter_name is not None:
        inc_dict_val(target, counter_name, counter_delta)

actions['plantationSettlement'] = plantationSettlement
# print_diff(game_state, plantationSettlement(game_state, **next_step['args']))
# next_step = get_step(game_log, 3, 0)

In [ ]:
def discardPlantations(state, **args):
    return state

actions['discardPlantations'] = discardPlantations

In [ ]:
def drawPlantations(state, plantations_to_set, **args):
    new_state = alter_dict(state)
    new_state['table']['plantations'] = setup_plantations(plantations_to_set)
    return new_state

actions['drawPlantations'] = drawPlantations
# print_diff(game_state, drawPlantations(game_state, **next_step['args']))

In [ ]:
def buildingBought(state, **kvargs):
    new_state = alter_dict(state)
    pl = get_player(new_state, **kvargs)
    add_building_to_player(pl, **kvargs)
    remove_building_from_table(new_state, **kvargs)
    remove_money_from_player(pl, **kvargs)
    add_money_to_bank(new_state, **kvargs)
#     display(JSON(new_state['table']), JSON(pl), JSON(kvargs))
    return new_state

def add_building_to_player(player, bld_type_tr, score_delta, bld_id, **kvargs):
    b = building_lookup[bld_id]
    inc_dict_value(player['slots'], 'buildings', b['tile_size'])
    inc_dict_value(player, 'building_vp', score_delta)
    inc_dict_value(player['potential_vp'], b['type'], b['potential_vp'])
    if b['type'] == 'commercial':
        player['buildings']['commercial'][b['name']] = False  
    else:
        inc_dict_value(player['buildings']['factories'][b['res_type']], 'jobs', b['jobs'])

def remove_building_from_table(state, bld_type_tr, **kvargs):
    inc_dict_value(state['table']['buildings'], bld_type_tr, -1)
    
def remove_money_from_player(player, cost:int, **kvargs):
    inc_dict_value(player, 'money', -int(cost))
    
def add_money_to_bank(state, cost, **kvargs):
    inc_dict_value(state['table'], 'money', cost)
    
actions['buildingBought'] = buildingBought
# pprint(get_diff(game_state, buildingBought(game_state, **next_step['args'])), width=10)

In [ ]:
def colonistsEarnedFromShip(state, delta, **kvargs):
    new_state = alter_dict(state)
    pl = get_player(new_state, **kvargs)
    inc_dict_value(pl['workers'], 'total', delta)
    inc_dict_value(pl['workers'], 'unemployed', delta)
    inc_dict_value(new_state['table']['workers'], 'arriving', -delta)
    return new_state

def colonistsEarnedFromSupply(state, delta, **kvargs):
    new_state = alter_dict(state)
    pl = get_player(new_state, **kvargs)
    inc_dict_value(pl['workers'], 'total', delta)
    inc_dict_value(pl['workers'], 'unemployed', delta)
    inc_dict_value(new_state['table']['workers'], 'available', -delta)
    return new_state

actions['colonistsEarnedFromSupply'] = colonistsEarnedFromSupply
actions['colonistsEarnedFromShip'] = colonistsEarnedFromShip

# pprint(get_diff(game_state, colonistsEarnedFromSupply(game_state, **next_step['args'])), width=10)

In [ ]:
def colonistToBuilding(state, bld_id, delta, **kvargs):
    new_state = alter_dict(state)
    b = datas['buildings'][bld_id]
    pl = get_player(new_state, **kvargs)
    pb = pl['buildings'][b['res_type']]

    if b['res_type'] != 'commercial':
        inc_dict_value(pb[b['res_type']], 'workers', delta)
    else:
        pb[b['name']] = True
    inc_dict_value(pl['workers'], 'unemployed', -delta)
    return new_state

def colonistToPlantation(state, bld_id, delta, **kvargs):
    new_state = alter_dict(state)
    pl = get_player(new_state, **kvargs)
    pp
    b = datas['buildings'][bld_id]
    if b['res_type']:
        inc_dict_value(pl['buildings']['factories'][b['res_type']], 'workers', delta)
    else:
        pl['buildings']['commercial'][b['bld_type']] = True
    inc_dict_value(pl['workers'], 'unemployed', -delta)
    return new_state

actions['colonistToBuilding'] = colonistToBuilding
actions['colonistToPlantation'] = colonistToPlantation
# pprint(get_diff(game_state, colonistToBuilding(game_state, **next_step['args'])), width=10)


# ACTIVE DEVELOPMENT ZONE


In [ ]:
def show_state(state):
    print('move {move_id}.{sub_move_id}'.format(**state))
    display(JSON(state))
#     pprint(state, depth=4, width=width)
#     for k,p in state['players'].items():
#         show_player(p) 
    
def show_player(pl):
    msg = """[{score}] {_} 
        goods:       {goods}
        plantations: {fields}
        buildings:   {buildings}"""
    
    print(msg.format(**pl))

# show_state(game_state)

In [ ]:
new_game_state = 1

In [ ]:
while new_game_state and len(game_log) > game_state['move_id']:
        
    next_step = get_step(game_log, **game_state)
    if step['type'] != 'gameStateChange':
        pprint(next_step, width=width)
        print('.'*width)
        print()
        
    new_game_state = apply_step(game_state, next_step)
    
    if new_game_state:
        if not game_state is new_game_state:
            print()    
            print_diff(game_state, new_game_state, consumer=lambda l: pprint(l, width=10))
            print('-'*width)    
            
        game_state = move_to_next_step(game_log, new_game_state)
        if not game_state['sub_move_id']: 
            print()
            print('='*width)
            show_state(game_state)
#             break          
            print('='*width)

In [ ]:
def get_active_player(game_state, step):
    if step['args']['type'] == 'activeplayer':
        player_id = step['args']['active_player']
        return game_state['players'][player_id]['_']
    else:
        return None

In [ ]:
for step in game_log[game_state['move_id']]['data']:
    if step['type'] == 'gameStateChange':
        print_log('> gameStateChange: ' + step['args']['description'], actplayer = get_active_player(game_state, step), **step['args'])
    elif step['type'] in ('updateReflexionTime', 'gameStateMultipleActiveUpdate'):
        print_log(step['type'] + ': ' + step['log'])
    else: 
        print_log(step['type'] + ': ' + step['log'], **dict(step['args']))
        print()
        display(JSON(step['args'], expanded=False))
    print('-' * width)

In [ ]:
def steps(log):
    for move_id, move in enumerate(log):
        for sub_move_id, sub_move in enumerate(move['data']):
#             if sub_move_id != 0:
                yield sub_move

In [ ]:
c = collections.Counter([s['type'] for s in steps(game_log) if s['type'] not in actions])
c.most_common(20)

In [ ]:
c = collections.Counter([s['args']['name']+str([k for k in s['args'].keys() if k not in exclude]) for s in steps(game_log) if s['type'] == 'gameStateChange'])
c.most_common(30)

In [ ]:
exclude = {'name', 'description', 'descriptionmyturn', 'type', 'action', 'active_player', 'args', 'reflexion', 'transitions', 'updateGameProgression', 'possibleactions'}
# JSON(sorted({str(s['args']['possibleactions']) for s in steps(game_log) if s['type'] == 'gameStateChange' and 'possibleactions' in s['args'].keys()}), expanded=False)
for s in steps(game_log):
    if s['type'] == 'gameStateChange':
        print_log(log, **dict(s['args']))

## VISUALIZATION

In [ ]:
planets = [
    ['Planet', 'Mass (kg)', 'Diameter (km)'],
    ['Mercury', 3.3022E23, 4879], 
    ['Venus', 4.896E24, []],
    ['Ear111th', 5.972E24, '12735'],
    ['Mars', 6.4191E23, 6772]];

#### ipy_table

In [ ]:
%%bash
pip install ipy_table

In [ ]:
from ipy_table import *
make_table(planets)
apply_theme('basic')

#### tabulate

In [ ]:
%%bash
pip install tabulate

In [ ]:
from IPython.display import HTML, display
import tabulate
table = [["Sun",696000,1989100000],
         ["Earth",6371,5973.6],
         ["Moon",1737,73.5],
         ["Mars",3390,641.85]]
display(HTML(tabulate.tabulate(table, tablefmt='html')))

#### merge html tables

In [ ]:
from IPython.display import HTML

def as_table(*data, repr=lambda s: s._repr_html_()):
    return HTML(
        '<table><tr>{}</tr></table>'.format(
            '</tr><tr>'.join(
                '<td>{}</td>'.format(
                    '</td><td>'.join(repr(_) for _ in row)
                ) for row in data
            )
        ))

In [ ]:
as_table(
    [1,2,3],
    [4,5,6],
    [7,8,9],
    repr=str
)

In [ ]:
x1 = make_table(planets)
x2 = make_table(planets)
apply_theme('basic')
as_table([x1, x2])

#### Pandas

In [ ]:
import pandas as pd
from IPython.display import JSON

### game visualizations

In [ ]:
def show_goods(player, **kvargs):
    return pd.Series(player['goods'], index=good_types).fillna(0).astype('int64')
# show_goods(player_example)

In [ ]:
def show_plantations(player, **kvargs):
    data = player['plantations']
    return pd.DataFrame(data, columns=good_types, index=['workers', 'jobs']).fillna(0).astype(str).apply(' / '.join)
# show_plantations(player_example)

In [ ]:
def show_factories(player, **kvargs):
    data = player['buildings']['factories']
    return pd.DataFrame(data, columns=good_types, index=['workers', 'jobs']).fillna(0).astype(str).apply(' / '.join)
# show_factories(player_example)

In [ ]:
def show_farming(player, **kvargs):
    g = show_goods(player, **kvargs)
    p = show_plantations(player, **kvargs)
    f = show_factories(player, **kvargs)
    return pd.DataFrame([g,p,f], index=['goods', 'plantations', 'factories'])

# show_farming(player_example)

In [ ]:
def show_commercial(player, **kvargs):
    data = [(k,'X' if v else '') for k,v in player_example['buildings']['commercial'].items()]
    return pd.DataFrame(data, columns=['name', 'active']).set_index(['name'])

show_commercial(player_example)

In [ ]:
def show_score(player, **kvargs):
    fmt = "{name}: $ {money}, [{delivery_vp} + {building_vp}]"
    return fmt.format(**player)

def calc_potential_vp(player, **kvargs): 
    return {
        'guild_hall': player['potential_vp']['factories'],
        'customs_house': player['delivery_vp']/4,
        'residence': max(4, player['slots']['plantations']-5),
        'city_hall': player['potential_vp']['commerial'],
        'fortress': player['workers']['total']/3
    }

In [ ]:
def show_player(player, **kvargs):
    s = show_score(player)
    c = show_commercial(player)
    f = show_farming(player)
    return as_table([],[HTML('<b>'+s+'</b>') ,HTML('')],[c, f.T])
# show_player(player_example)

### Old output

In [ ]:
def fixed_len(s,l):
    raise Error('Obsolete')
#     return str(s + ' '*(l-len(s)) if l > len(s) else s)

def toarray(d):
    return [int(d[str(i+1)]) for i in range(4)]

In [ ]:
import itertools

def transpose(*a):
    return list(map(list, itertools.zip_longest(*a, fillvalue='')))

def fill_columns(*cols, **kwargs):
    to_len = lambda a: [len(v) for v in a]
    max_len= lambda a: max(to_len(a))

    row_count = max_len(cols)
    cols = [c + ['' for i in range(row_count-len(c))] for c in cols]
    return cols

print(transpose(['1'+'.'*2], [1,2,3,'4'+'.'*7], [1,2,'3'+'.'*6]))

In [ ]:
def pad_columns(*rows, **kwargs):
    separator = kwargs.get('separator',' ')
    to_col = lambda aa,i: [a[i] for a in aa]
    to_str = lambda a: [str(v) for v in a]
    to_len = lambda a: [len(v) for v in a]

    widths = [max(to_len(to_str(to_col(rows, i)))) for i in range(len(rows[0]))]
    ptr = separator.join(['{:^%s}'%s for s in widths])
    return [ptr.format(*r) for r in rows]

def print_lines(lines):
    print('\n'.join(lines))
    
    
# f = fill_columns
# t = transpose
p = [['n'+'.'*2], ['i1','i2','i3','i4'+'.'*7], ['t1','t2','t3'+'.'*6]]    
n = ['N','I','T']
c = [n] + t(*f(*p))
# filled = t(*c)
# pprint(filled)
# print
print_lines(pad_columns(*c))


In [ ]:
def make_tabular(cols, data):
    mapper = lambda p: [p[c] for c in cols]
    pp = [cols]
    filler = [['' for i in range(len(cols))]]
    for p in data:
        pp += filler + transpose(*fill_columns(*mapper(p)))
    return pp

In [ ]:
def show_skills(a):
    return "{a[0]}{a[1]}{a[2]}{a[3]}".format(a=a)

def show_inventor(i):
    return "{sign} {_:20} {skills_f} / {objective_f} = {points}".format(
        skills_f = show_skills(i['skills']),
        objective_f = show_skills(i['objective']),
        sign='-' if not i.get('active', True) else '+', 
        **i)

def show_inventors(p):
    return [show_inventor(get_inventor(p, inventor_no=i)) for i in xrange(1,5)]

def show_token(t):
    return "{type_name:.7}({type_arg})".format(**t)
    
def show_tokens(p):
    return sorted([show_token(t) for id,t in p['tokens'].items()])

def show_player(p):
    return dict( 
        name = ['{_:10.10}'.format(**p)],
        score = [p['score']],
        tokens=show_tokens(p),
        inventors=show_inventors(p))

def show_players(s):
    cols = ['name','score','inventors','tokens']
    data = [show_player(p) for p in s['players'].values()]
    return make_tabular(cols, data)
    
def show_player_names(s,c):
    return [ '{}: {}'.format(s['players'][id]['_'], p) for id,p in sorted(c.items(), key=lambda k: -k[1])]

def show_invention(i):
    return dict(
        name=['{_:15}'.format(**i), '{} / {} '.format(show_skills(i['cur']), show_skills(i['cost']))],
        bidders=show_player_names(game_state, i['players']),
        rewards=show_tokens(i) + ['card#{serienbr} ({points})'.format(**i) if not i.get('taken', False) else ''])
                      
def show_inventions(s):
    cols = ['name','rewards','bidders']
    data = [show_invention(i) for i in s['table'].values()]
    return make_tabular(cols, data)
    
def show_state(s):
    parts = [
        pad_columns(*show_players(game_state),    separator=' | '),
        pad_columns(*show_inventions(game_state), separator=' | ')
    ]
    
    print_lines(pad_columns(*transpose(*fill_columns(*parts)), separator='  ##  '))
    
# print_lines(pad_columns(*show_players(game_state), separator=' | '))
print_lines(pad_columns(*show_inventions(game_state), separator=' | '))
# show_state(game_state)

### Old stuff






In [ ]:
def useClassificationToken(state, number, **args):
    state = alter_dict(state)
    p = get_player(state, **args)
    add_card(p, int(number), **args)
    pop_token(p, **args)
    return state

actions['useClassificationToken'] = useClassificationToken
#print_diff(useClassificationToken(game_state, **step['args']))

In [ ]:
def tableWindow(state, **args):
    state = alter_dict(state)
    table = alter_dict(args)['table']
#     pprint(table)
    last = len(table)-1
    for i in xrange(1,len(table[0])):
        table[0][i] = table[0][i]['args']['player_name']
        table[last][i] = int(table[last][i][3:-4])
    print_lines(pad_columns(*table, separator=' | '))
    return state

actions['tableWindow'] = tableWindow
#print_diff(apply_step(game_state, step))

In [ ]:
def score(state, score, **args):
    state = alter_dict(state)
    
    p = get_player(state, **args)
    p['score'] = score

    print_log('{p[_]} scores {score}'.format(score=score, p=p))
    return state

actions['score'] = score
#print_diff(score(game_state, **step['args']))

In [ ]:
def refresh(state, **args):
    state = alter_dict(state)
    
    p = get_player(state, **args)
    for i in xrange(1,5):
        i = get_inventor(p, i, **args)
        i['active'] = True
    
    return state

actions['refresh'] = refresh
#print_diff(refresh(game_state, **step['args']))

In [ ]:
def endRewards(state, invention_id, **args):
    print_log('endRewards')
    state = alter_dict(state)
    state['table'].pop(invention_id)
    return state
    
actions['endRewards'] = endRewards

In [ ]:
def increaseSkill(state, **args):
    state = alter_dict(state)

    p = get_player(state, **args)
    i = get_inventor(p, **args)
    pop_token(p, **args)
    set_skill(i, **args)

    return state
    
actions['increaseSkill'] = increaseSkill

In [ ]:
def pickReward(state, **args):
    state = alter_dict(state)
    p = get_player(state, **args)
    i = get_invention(state, **args)
    
    remove_bidder(state, i, **args)
    if step['args']['is_card']:
        return pick_card(state, p, i, **args)
    else:
        return pick_token(state, p, i, **args)

def pick_token(state, p, i, rewardcard, **args):
    token_id = rewardcard['id']
    r = pop_token(i, token_id, **args)
    add_token(p, token_id, r, **args)
    return state
    
def pick_card(state, p, i, serie, **args):
    add_card(p, serie, **args)
    i['taken'] = True
    return state

def remove_bidder(state, i, player_id, **args):
    i['players'].pop(player_id)
    

actions['pickReward'] = pickReward
print_diff(pickReward(game_state, **step['args']))

In [ ]:
def work(state, **args):
    state = alter_dict(state)
    deactivate_inventor(state, **args)
    place_cubes(state, **args)
    return state

def deactivate_inventor(state, inventor, **args):
    if inventor:
        p = get_player(state, **args)
        i = get_inventor(p, inventor['no'], **args)
        i['active'] = False

def place_cubes(state, player_id, placed, nbr, **args):    
    invention = get_invention(state, **args)
    
    for k,v in placed.items():
        invention['cur'][int(k)-1] += int(v)

    inc_dict_val(invention['players'], player_id, nbr)    

actions['work'] = work
print_diff(work(game_state, **step['args']))

In [ ]:
def startAge(state, **args):
    l = dict()
    initInventions(datas, l, **args)
    initTokens(datas, l, **args)
    return alter_dict(state, age=args['age'], table=l)

def initInventions(datas, l, inventions, **args):
    for v in inventions:
        inv = datas['invention_types'][v['type']]
        d = map_dict(inv, 'cost', 'points', 'serienbr', _='name')
        d.update(map_dict(v, 'id', 'type'))
        d.update(tokens={}, cur=[0,0,0,0], players={}, cost=toarray(d['cost'])) 
        l[v['id']] = d

def initTokens(datas, l, inventions_tokens, **args):
    for v in inventions_tokens:
        inv = datas['token_types'][v['type']]
        d = map_dict(v, 'type', 'type_arg')
        d.update(type_name = inv['type'])

        l[v['location_arg']]['tokens'][v['id']] = d        
        
actions['startAge'] = startAge        
#print_diff(startAge(game_state, **step['args']))

In [ ]:
def get_invention(c, invention_id, **args):
    return c['table'][invention_id]

def get_inventor(c, inventor_no, **args):
    return c['inventors'][inventor_no]

def get_player(c, player_id, **args):
    return c['players'][player_id]

def pop_token(c, token_id, **args):
    return c['tokens'].pop(token_id)

def add_token(c, token_id, token, **args):
    c['tokens'][token_id] = token
    
def set_skill(c, skill, value, **args):
    c['skills'][int(skill)-1] = value    
    
def add_card(c, card, **args):
    c['cards'][card] += 1        